# 3.1 Advanced Prompt Caching

**Duration**: ~45 minutes

---

## 1. How Prompt Caching Works

Prompt caching stores frequently used portions of prompts to avoid reprocessing.

| Term | Description |
|------|-------------|
| **Cache Write** | First occurrence (1.25x cost) |
| **Cache Read** | Reuse cached content (0.1x cost = 90% savings) |
| **TTL** | 5 minutes (resets on hit) |
| **Min Tokens** | 1,024 tokens (content before checkpoint must meet this) |
| **Max Checkpoints** | 4 per request |

**Important**: If your content doesn't meet the minimum token threshold, the checkpoint is ignored and no caching occurs.

> [Docs](https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-caching.html)

## 2. Setup

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os, boto3

bedrock_runtime = boto3.client('bedrock-runtime', region_name=os.getenv('AWS_DEFAULT_REGION', 'us-east-1'))
MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"
print(f"Model: {MODEL_ID}")

### Cache Metrics Helper

The Bedrock API returns cache metrics in the `usage` field:
- `inputTokens`: Regular input tokens (not cached)
- `outputTokens`: Generated output tokens
- `cacheWriteInputTokens`: Tokens written to cache (first occurrence)
- `cacheReadInputTokens`: Tokens read from cache (subsequent hits)

**Pricing** (per 1M tokens, see [Bedrock Pricing](https://aws.amazon.com/bedrock/pricing/)):

| Token Type | Cost Multiplier |
|------------|----------------|
| Regular input | 1.0x base price |
| Cache write | 1.25x base price |
| Cache read | 0.1x base price (90% savings) |

In [ ]:
def extract_cache_metrics(response):
    usage = response.get('usage', {})
    return {
        'input_tokens': usage.get('inputTokens', 0),
        'output_tokens': usage.get('outputTokens', 0),
        'cache_write': usage.get('cacheWriteInputTokens', 0),
        'cache_read': usage.get('cacheReadInputTokens', 0)
    }

def print_cache_metrics(metrics, label=None):
    header = f"Request {label}" if label else "Cache Metrics"
    print(f"\n{'='*50}")
    print(header)
    print(f"{'='*50}")
    print(f"Input tokens:  {metrics['input_tokens']:,}")
    print(f"Output tokens: {metrics['output_tokens']:,}")
    print(f"Cache write:   {metrics['cache_write']:,}")
    print(f"Cache read:    {metrics['cache_read']:,}")

def calculate_savings(metrics_list, input_price=3.0):
    total_input = sum(m['input_tokens'] for m in metrics_list)
    total_write = sum(m['cache_write'] for m in metrics_list)
    total_read = sum(m['cache_read'] for m in metrics_list)
    
    cost_cached = (
        (total_input / 1_000_000) * input_price +
        (total_write / 1_000_000) * input_price * 1.25 +
        (total_read / 1_000_000) * input_price * 0.1
    )
    total_tokens = total_input + total_write + total_read
    cost_no_cache = (total_tokens / 1_000_000) * input_price
    
    savings_pct = ((cost_no_cache - cost_cached) / cost_no_cache * 100) if cost_no_cache > 0 else 0
    hit_rate = (total_read / (total_read + total_write) * 100) if (total_read + total_write) > 0 else 0
    
    return {'hit_rate': hit_rate, 'savings_pct': savings_pct, 'cache_write': total_write, 'cache_read': total_read}

print("Helper functions defined")

## 3. Pattern 1: Simple Message Caching

Single checkpoint for document Q&A.

**Use case**: Multiple questions about the same document.

**Note**: The document must be at least 1,024 tokens for caching to activate.

In [ ]:
# Create a document large enough to cache (>1024 tokens)
PRODUCT_MANUAL = '''
# CloudCommerce Smart Home Hub - Complete Product Manual and Technical Documentation

## Executive Overview
The CloudCommerce Smart Home Hub represents the next generation of home automation technology.
This central control unit seamlessly manages all your smart home devices through a unified interface.
It supports multiple wireless protocols including WiFi, Zigbee, Z-Wave, and Bluetooth for maximum device compatibility.

## Detailed Technical Specifications
- Processor: Quad-core ARM Cortex-A53 @ 1.8GHz with dedicated security enclave
- Memory: 2GB DDR4 RAM with ECC support for reliable operation
- Storage: 16GB eMMC flash storage with wear leveling
- WiFi: Dual-band 802.11ax (WiFi 6) with MU-MIMO support
- Bluetooth: Version 5.0 with Bluetooth Low Energy (BLE) mesh networking
- Zigbee: 3.0 certified with Green Power support for battery-free devices
- Z-Wave: Plus V2 certified with S2 security framework
- Power: 5V/3A USB-C with Power Delivery support
- Dimensions: 120mm x 120mm x 35mm (compact design)
- Weight: 280g
- Operating Temperature: 0C to 40C
- Humidity: 10% to 90% non-condensing

## Comprehensive LED Status Indicator Guide
The hub features a multi-color LED ring that provides instant status feedback:
- Solid Blue: Normal operation - hub is connected and functioning properly
- Blinking Blue: Pairing mode - hub is ready to connect new devices
- Solid Green: Processing command - hub is executing an automation or command
- Blinking Green: Firmware download in progress
- Solid Yellow: Warning state - check app for details
- Blinking Yellow: Firmware update being installed
- Solid Red: Error condition - restart required
- Blinking Red: Network connection lost - check your router
- Purple Pulse: Voice assistant active and listening
- White Breathing: Night mode enabled

## Complete Setup Instructions
### Initial Hardware Setup
1. Carefully unbox the hub and verify all components are present
2. Place the hub in a central location for optimal wireless coverage
3. Connect the included USB-C power adapter to the hub
4. Wait for the LED to turn solid blue (approximately 2-3 minutes for initial boot)

### Software Configuration
5. Download the CloudCommerce app from the Apple App Store or Google Play Store
6. Create a new account or sign in with your existing credentials
7. Tap the Add Device button and select Smart Home Hub from the device list
8. Follow the in-app instructions to connect the hub to your WiFi network
9. The hub will automatically check for and install any available firmware updates
10. Once setup is complete, you can begin adding smart devices to your network

## Extensive Troubleshooting Guide

### Hub Won Not Connect to WiFi
This is one of the most common issues during initial setup. Follow these steps:
1. Verify your router is broadcasting on the 2.4GHz band (required for initial setup)
2. Ensure you are entering the correct WiFi password (case-sensitive)
3. Move the hub closer to your router during initial setup
4. Restart your router and wait 2 minutes before trying again
5. Check that your router firmware is up to date
6. Disable any MAC address filtering temporarily
7. If all else fails, factory reset the hub by holding the reset button for 10 seconds

### Smart Devices Not Responding
When devices stop responding to commands, try these solutions:
1. Check that the device is within range (Zigbee: 10m indoors, Z-Wave: 30m, WiFi: varies)
2. Verify the device battery level if it is a battery-powered device
3. Check for physical obstructions between the hub and device
4. Remove and re-add the device in the app
5. Check for firmware updates for both the hub and the device
6. Ensure no wireless interference from other 2.4GHz devices

### App Shows Hub Offline
If the app indicates the hub is offline:
1. Verify your smartphone has an active internet connection
2. Check the hub LED status indicator
3. Restart the hub by unplugging power for 30 seconds
4. Ensure your phone and hub are on the same local network
5. Check if your router firewall is blocking the hub
6. Try force-closing and reopening the app

## Voice Assistant Integration Guide
The hub supports seamless integration with major voice assistants:

### Amazon Alexa Setup
1. Open the Amazon Alexa app on your smartphone
2. Navigate to Skills and Games
3. Search for CloudCommerce Smart Home
4. Tap Enable and sign in with your CloudCommerce account
5. Say Alexa, discover devices to find your smart home devices

### Google Assistant Setup
1. Open the Google Home app
2. Tap the plus icon and select Set up device
3. Choose Works with Google and search for CloudCommerce
4. Link your CloudCommerce account
5. Your devices will automatically appear in the Google Home app

### Apple HomeKit Setup
1. Open the Apple Home app on your iOS device
2. Tap Add Accessory
3. Scan the HomeKit code on the bottom of your hub
4. Follow the prompts to add the hub to your home

## Warranty and Support Information
This product comes with a comprehensive 2-year limited warranty covering all manufacturing defects.
The warranty does not cover damage resulting from misuse, accidents, unauthorized modifications, or normal wear.
For warranty claims or technical support, contact support@cloudcommerce.example with your proof of purchase.
Extended warranty options are available for purchase within 30 days of your original purchase.

## Safety and Regulatory Information
Please read all safety information before using this product:
- Do not expose the hub to water, rain, or extreme humidity
- Operating temperature range: 0C to 40C (32F to 104F)
- Do not block the ventilation holes on the bottom of the device
- Use only the provided power adapter or a certified USB-C PD adapter
- Keep the hub away from heat sources and direct sunlight
- Do not attempt to open or repair the hub yourself

## Regulatory Compliance
FCC ID: 2AXXXXXXXXX
IC: XXXXX-XXXXXXX
This device complies with Part 15 of the FCC Rules and Industry Canada license-exempt RSS standards.
Operation is subject to the following two conditions:
(1) This device may not cause harmful interference, and
(2) This device must accept any interference received, including interference that may cause undesired operation.
'''

print(f"Document: ~{len(PRODUCT_MANUAL.split())} words (~{len(PRODUCT_MANUAL)//4} tokens estimated)")

In [ ]:
# Request 1: Cache WRITE
r1 = bedrock_runtime.converse(
    modelId=MODEL_ID,
    messages=[{"role": "user", "content": [
        {"text": PRODUCT_MANUAL},
        {"cachePoint": {"type": "default"}},
        {"text": "What are the LED status colors?"}
    ]}],
    inferenceConfig={"maxTokens": 200}
)
m1 = extract_cache_metrics(r1)
print_cache_metrics(m1, 1)
print(f"\nAnswer: {r1['output']['message']['content'][0]['text'][:200]}...")

In [ ]:
# Request 2: Cache READ (same document, different question)
r2 = bedrock_runtime.converse(
    modelId=MODEL_ID,
    messages=[{"role": "user", "content": [
        {"text": PRODUCT_MANUAL},
        {"cachePoint": {"type": "default"}},
        {"text": "How do I factory reset the hub?"}
    ]}],
    inferenceConfig={"maxTokens": 200}
)
m2 = extract_cache_metrics(r2)
print_cache_metrics(m2, 2)
print(f"\nAnswer: {r2['output']['message']['content'][0]['text'][:200]}...")

In [ ]:
# Calculate savings across both requests
savings = calculate_savings([m1, m2])
print("\nCACHE PERFORMANCE SUMMARY")
print("=" * 40)
print(f"Cache write: {savings['cache_write']:,} tokens")
print(f"Cache read:  {savings['cache_read']:,} tokens")
print(f"Hit rate:    {savings['hit_rate']:.1f}%")
print(f"Savings:     {savings['savings_pct']:.1f}%")

## 4. Pattern 2: Multi-Checkpoint Caching

For agentic workflows: **Tools -> System -> Messages**

Each checkpoint caches all content up to that point. The `cachePoint` must be a separate item in the array.

**Note**: Each checkpoint requires minimum 1,024 tokens of cumulative content before it.

In [ ]:
# Tools with detailed descriptions to meet token minimum
TOOLS = [
    {"toolSpec": {
        "name": "search_products",
        "description": "Search the product catalog by name, category, SKU, or keywords. Returns matching products with full details including prices, availability, specifications, customer ratings, and related products. Supports filtering by price range, brand, category, and availability status. Results are paginated with customizable page size. " * 10,
        "inputSchema": {"json": {"type": "object", "properties": {"query": {"type": "string"}, "category": {"type": "string"}, "max_price": {"type": "number"}}, "required": ["query"]}}
    }},
    {"toolSpec": {
        "name": "check_inventory",
        "description": "Check real-time inventory levels for a specific product SKU across all warehouse locations. Returns quantity available, reserved quantity, incoming shipments, and estimated restock dates. Also provides inventory history and trend analysis for demand forecasting. " * 10,
        "inputSchema": {"json": {"type": "object", "properties": {"sku": {"type": "string"}, "warehouse_id": {"type": "string"}}, "required": ["sku"]}}
    }},
    {"cachePoint": {"type": "default"}}  # Checkpoint after tools
]

# System prompt with enough content
SYSTEM = [
    {"text": "You are a helpful and knowledgeable customer support assistant for CloudCommerce, a leading e-commerce platform specializing in electronics and smart home devices. Your role is to assist customers with product inquiries, order status, returns, and technical support. Always be professional, empathetic, and solution-oriented. Use the available tools to look up accurate product and inventory information. Never make up information - if you don't know something, say so and offer to help find the answer. Follow all company policies regarding returns, refunds, and warranties. " * 5},
    {"cachePoint": {"type": "default"}}  # Checkpoint after system
]

print(f"Tools: {len([t for t in TOOLS if 'toolSpec' in t])} defined")
print(f"System prompt configured with cache checkpoint")

In [ ]:
# Request with multi-checkpoint caching
r3 = bedrock_runtime.converse(
    modelId=MODEL_ID,
    toolConfig={"tools": TOOLS},
    system=SYSTEM,
    messages=[{"role": "user", "content": [{"text": "Do you have any wireless keyboards in stock?"}]}],
    inferenceConfig={"maxTokens": 200}
)
print_cache_metrics(extract_cache_metrics(r3), "3 (Multi-checkpoint WRITE)")

In [ ]:
# Second request - should get cache READ
r4 = bedrock_runtime.converse(
    modelId=MODEL_ID,
    toolConfig={"tools": TOOLS},
    system=SYSTEM,
    messages=[{"role": "user", "content": [{"text": "What about mechanical keyboards?"}]}],
    inferenceConfig={"maxTokens": 200}
)
print_cache_metrics(extract_cache_metrics(r4), "4 (Multi-checkpoint READ)")

## 5. Pattern 3: Simplified Caching (Claude)

For Claude models, place ONE checkpoint at the end of static content.

Bedrock automatically checks back ~20 content blocks for cache matches.

**Benefit**: Simpler implementation, automatic prefix matching.

In [ ]:
# Using same TOOLS and SYSTEM from Pattern 2
# The single checkpoint at end of SYSTEM covers everything
r5 = bedrock_runtime.converse(
    modelId=MODEL_ID,
    toolConfig={"tools": TOOLS, "toolChoice": {"auto": {}}},
    system=SYSTEM,
    messages=[{"role": "user", "content": [{"text": "Check inventory for SKU-KB-MECH-001"}]}],
    inferenceConfig={"maxTokens": 200}
)
print_cache_metrics(extract_cache_metrics(r5), "5 (Simplified caching)")

## 6. Design Principles

### Principle 1: Longest Cacheable Prefix
```
GOOD: [Static Tools] -> [Static System] -> [Dynamic Query]
BAD:  [Dynamic Query] -> [Static System] -> [Static Tools]
```

### Principle 2: Separate Static from Dynamic
| Static (Cache) | Dynamic (Don't Cache) |
|----------------|----------------------|
| System prompts | User queries |
| Tool definitions | Conversation history |
| Reference docs | Real-time data |

### Principle 3: Monitor Cache Performance
- **Hit rate**: `cache_read / (cache_read + cache_write)` - Target >70%
- **Cost savings**: Compare cached vs non-cached costs

### Principle 4: Cache Warm-up
Pre-warm caches during low-traffic periods.

## 7. CRIS with Caching

- Cache is region-specific
- Global CRIS may route to different regions = cache misses
- Geographic CRIS (us., eu.) keeps requests in same region pool

**Recommendation**: Use geographic profiles (e.g., `us.anthropic.claude-*`) for consistent caching.

## Summary

| Pattern | Checkpoints | Use Case |
|---------|-------------|----------|
| **Simple** | 1 (message) | Document Q&A |
| **Multi** | 2-4 (tools->system->history) | Agentic workflows |
| **Simplified** | 1 (auto-matching) | Claude models |

**Best Practices**:
1. Min 1,024 tokens per checkpoint
2. Static content first
3. Monitor hit rate (target >70%)
4. Consider TTL (5 min) for request patterns

---

**Next**: Apply these patterns in the Developer Journey module.